In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 9.0)

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression, HuberRegressor, SGDRegressor
from sklearn.cross_decomposition import PLSRegression
from sklearn.decomposition import PCA
# from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin
from sklearn.pipeline import Pipeline
from datetime import datetime
# import lightgbm as lgb

import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.mlab as mlab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16.0, 9.0)

In [2]:
n_inputs = 920
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return x, y

In [64]:
n_inputs = 920
def preprocess_mean(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs)
    x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    y_mean = np.mean(y)
    return y_mean

In [3]:
def csv_reader_dataset_train(filepaths, repeat=1, n_readers=13,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)

In [75]:
def csv_reader_dataset_train_mean(filepaths, repeat=1, n_readers=13,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess_mean, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset

In [33]:
def csv_reader_dataset_val(filepaths, repeat=1, n_readers=9,
                       n_read_threads=None, shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)

In [49]:
def csv_reader_dataset_test(filepaths, repeat=1, n_readers=1,
                       n_read_threads=None, shuffle_buffer_size=1,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)

In [6]:
train_filepaths = "../../920data/train/df_%s.csv"

valid_filepaths = "../../920data/val/df_*.csv"

test_filepaths = "../../920data/df_1997.csv"

In [8]:
train_set = csv_reader_dataset_train(train_filepaths)

valid_set = csv_reader_dataset_val(valid_filepaths)


In [68]:
train_set

<DatasetV1Adapter shapes: ((None, 920), (None, 1)), types: (tf.float32, tf.float32)>

In [80]:
# train_mean = csv_reader_dataset_train_mean(train_filepaths)

In [50]:
test_set = csv_reader_dataset_test(test_filepaths)

In [15]:
nn1_model = keras.models.Sequential()
nn1_model.add(keras.layers.InputLayer(input_shape=[920]))
nn1_model.add(keras.layers.BatchNormalization())
nn1_model.add(keras.layers.Dense(32, activation='relu'))
nn1_model.add(keras.layers.BatchNormalization())
nn1_model.add(keras.layers.Dense(1))

In [16]:
nn1_model.compile(loss='mse',optimizer='adam')

In [18]:
nn1_model.fit(train_set, epochs=10, validation_data=valid_set)

Epoch 1/10
26788/26788 [==============================] - 350s 13ms/step - loss: 0.3163 - val_loss: 0.0000e+00
Epoch 2/10
26788/26788 [==============================] - 327s 12ms/step - loss: 0.3095 - val_loss: 0.3433
Epoch 3/10
26788/26788 [==============================] - 321s 12ms/step - loss: 0.3087 - val_loss: 0.3367
Epoch 4/10
26788/26788 [==============================] - 321s 12ms/step - loss: 0.3081 - val_loss: 0.3401
Epoch 5/10
26788/26788 [==============================] - 321s 12ms/step - loss: 0.3075 - val_loss: 0.3360
Epoch 6/10
26788/26788 [==============================] - 318s 12ms/step - loss: 0.3072 - val_loss: 0.3438
Epoch 7/10
26788/26788 [==============================] - 322s 12ms/step - loss: 0.3070 - val_loss: 0.3477
Epoch 8/10
26788/26788 [==============================] - 324s 12ms/step - loss: 0.3066 - val_loss: 0.3595
Epoch 9/10
26788/26788 [==============================] - 322s 12ms/step - loss: 0.3066 - val_loss: 0.3460
Epoch 10/10
26788/26788 [========

In [51]:
y_pred = nn1_model.predict(test_set)

In [56]:
y_pred.T

array([[-0.01991238, -0.2699778 , -0.3973542 , ..., -0.21251288,
        -0.06760653, -0.21158417]], dtype=float32)

In [57]:
y_pred1 = y_pred[0:10000].T

In [58]:
y_pred1

array([[-0.01991238, -0.2699778 , -0.3973542 , ..., -0.06563534,
        -0.16934478, -0.05159641]], dtype=float32)

In [22]:
filename = "../../920data/df_1997.csv"

In [23]:
import csv
with open(filename,'rt') as csvfile:
    reader = csv.DictReader(csvfile)
    column = [row['ret'] for row in reader]

In [60]:
y_true = np.array(column,dtype = 'float32')

In [61]:
y_true

array([-0.07082833, -0.8619448 , -0.889956  , ...,  0.73478264,
       -0.21605352, -0.42341137], dtype=float32)

In [62]:
y_true1 = y_ture[0:10000]

In [31]:
def r2_oos(y_true, y_pred):
    return 1 - np.sum((y_true - y_pred)**2) / np.sum(y_true**2)

In [63]:
r2_oos(y_true=y_true1, y_pred=y_pred1)

-0.301563024520874